### Importing Libraries

In [2]:
import os
import string
import csv
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.utils import shuffle
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import preprocessing
import codecs

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/varunchopra/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

<h3>Reading the input from lyrics file.</h3>

In [82]:
stop_words = (stopwords.words('english'))
newsw=['youve','youd','youll','shes','ive','hes','cant','never','dont','one','didnt']
stop_words.extend(newsw)

header = ['word', 'frequency', 'genre']

with open(f'Documents/genres2/freqDict.csv', 'w', newline = "") as file:
    writer = csv.writer(file)
    writer.writerow(header)

# a list of all the present genres
genresList = ['blues_lyrics', 'country_lyrics', 'disco_lyrics', 'hiphop_lyrics', 'metal_lyrics', 'pop_lyrics', 'reggae_lyrics', 'rock_lyrics']

# iterating over the list of all the genres in the genreList
for genreName in genresList:
    
    genreLyrics = ''
    lyricStopped = list()
    
    # getting the path of the current genre
    genre = os.listdir(f'Documents/genres2/{genreName}')
    
    # selecting each song from each genre
    for song in genre:
        path = f'Documents/genres2/{genreName}/{song}'
        
        if song == '.DS_Store':
            continue
        
        # defining the path of a song lyrics file
#         with codecs.open(file_name, 'r', encoding='utf-8',
#                  errors='ignore') as fdata:
        songname = open(path, 'r')
        
        # read all the words in lower case
        lyric = songname.read().lower()
        
        # removing punctuations
        for punct in string.punctuation:
            lyric = lyric.replace(punct, '')
        
        # adding lyrics to genreLyrics list
        genreLyrics += lyric
    
    # extracting words from a particular genreLyrics
    words = genreLyrics.split()
    
    # removing the stopwords
    for l in words:
        if l not in stop_words:
            lyricStopped.append(l)
    
    # frequency dictionary
    freqDict = {}
    
    # counting the occurence of all the words in the dictionary
    # no need to make a set of UNIQUE WORDS as the "keys" of this dictionary will all be unique
    for word in lyricStopped:
        freqDict[word] = freqDict.get(word, 0) + 1
    # print the dictionary
    with open(f'Documents/genres2/freqDict.csv', 'a', newline = "") as file:
        writer = csv.writer(file)
        for key, value in freqDict.items():
            writer.writerow([key, value, genreName.replace('_lyrics','')])


In [72]:
dataset = pd.read_csv('Documents/genres2/freqDict.csv')
X = dataset.iloc[:, [0,1]].values
y = dataset.iloc[:, 2].values

print(X[0])

['love' 56]


In [73]:
le = preprocessing.LabelEncoder()
X_encoded=le.fit_transform(X)
X_encoded = np.reshape(X_encoded,(-1,1))
print(X_encoded)

ValueError: bad input shape (29143, 2)

In [58]:
le = preprocessing.LabelEncoder()
y_encoded=le.fit_transform(y)
y_encoded = np.reshape(y_encoded,(-1,1))
print(y_encoded)

[[0]
 [0]
 [0]
 ...
 [7]
 [7]
 [7]]


In [59]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_encoded, test_size = 0.25, random_state = 0)

In [60]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train,y_train)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GaussianNB(priors=None, var_smoothing=1e-09)

In [61]:
y_pred = classifier.predict(X_test)

In [62]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[   0    0    0  697    0    0    0    0]
 [   0    0    0  694    0    0    0    0]
 [   0    0    0  824    0    0    0    0]
 [   0    0    0 2479    0    0    0    0]
 [   0    0    0  919    0    0    0    0]
 [   0    0    0  915    0    0    0    0]
 [   0    0    0  281    0    0    0    0]
 [   0    0    0  477    0    0    0    0]]


In [64]:
from sklearn import metrics

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.34024155915454296
